In [ ]:
# 전국 파스쿠치 매장 정보 수집

In [1]:
import pandas as pd
import requests
import time
from tqdm import trange
from bs4 import BeautifulSoup as bs

In [13]:
def get_url(page_no=1):
    url = f"https://www.caffe-pascucci.co.kr/store/storeList.asp?page={page_no}"
    return url

In [14]:
def get_last_page():
    "마지막 페이지 구하기"
    url = get_url()
    response = requests.get(url)
    html = bs(response.text)
    last_page = int(html.select("#container > div.contents > div.schResultWr.schStoreResult > div > ul > li.last1 > a")[0]['href'].split('=')[-1])
    return last_page

In [15]:
def get_one_page(page_no):
    "한 페이지 수집"
    # 종목 URL 만들기
    url = get_url(page_no)
    # requests
    response = requests.get(url)
    html = bs(response.text)
    # 데이터프레임 만들기
    store = html.select("#container > div.contents > div.schResultWr.schStoreResult > table > tbody > tr > td.storeName > strong")
    location = html.select("#container > div.contents > div.schResultWr.schStoreResult > table > tbody > tr > td.storeArea")
    address = html.select("#container > div.contents > div.schResultWr.schStoreResult > table > tbody > tr > td.subject > p.addr")
    
    return pd.DataFrame({'브랜드명':'파스쿠찌', '지점명': store, '시도명':location, '주소':address})

In [17]:
def get_all_pages():
    "모든 페이지 수집"
    last = get_last_page()
    page_list = []

    # 1페이지 ~ 끝페이지
    for page_num in trange(1, last+1):
        page = get_one_page(page_num)
        page_list.append(page)
        time.sleep(0.02)
    
    # 모든 페이지 하나의 데이터프레임으로 합치기
    df_all_page = pd.concat(page_list)
    # 결측치 제거
    df_all_page = df_all_page.dropna(how="all")
    # 인덱스 리셋
    df_all_page = df_all_page.reset_index(drop=True)
    
    df_all_page['지점명'] = df_all_page['지점명'].map(lambda x:str(x).split('>')[1].split('<')[0])
    df_all_page['시도명'] = df_all_page['시도명'].map(lambda x:str(x).split('>')[1].split('<')[0])
    df_all_page['주소'] = df_all_page['주소'].map(lambda x:str(x).split('>')[1].split('<')[0])
    return df_all_page


In [18]:
# 20230220 기준 522개 매장
df_all = get_all_pages()
df_all

100%|██████████| 53/53 [01:23<00:00,  1.58s/it]


,브랜드명,지점명,시도명,주소
0,파스쿠찌,고양향동스타허브점,경기도,"경기도 고양시 덕양구 꽃마을로 64, DMC스타허브플러스 1층"
1,파스쿠찌,대전중앙로점,대전,"대전광역시 중구 중앙로 126, 1층 (대흥동)"
2,파스쿠찌,세종다정중앙점,세종시특별자치시,세종특별시 다정중앙로 46
3,파스쿠찌,군산디오션시티점,전라북도,전라북도 군산시 백릉로 145
4,파스쿠찌,홈플러스아시아드점,부산,부산 연제구 거제동 1208 홈플러스
...,...,...,...,...
517,파스쿠찌,수원AK점,경기도,경기도수원시 팔달구 매산로1가 수원역사 AK PLAZA 3층
518,파스쿠찌,롯데마산점,경상남도,경상남도 창원시 마산합포구 신포동2가 28
519,파스쿠찌,광안리점,부산,부산시 수영구 민락동 179-2 파스쿠찌 광안리점
520,파스쿠찌,테크노마트점,서울,서울시 광진구 구의동 546-4 테크노마트 지하 D-10호 파스쿠찌 테크노마트점


In [19]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522 entries, 0 to 521
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   브랜드명    522 non-null    object
 1   지점명     522 non-null    object
 2   시도명     522 non-null    object
 3   주소      522 non-null    object
dtypes: object(4)
memory usage: 16.4+ KB


In [20]:
# 지점명이 중복된 행이 있다
df_all.nunique()

브랜드명      1
지점명     521
시도명      17
주소      522
dtype: int64

In [22]:
# 매장 하나가 두 번 들어간 것을 확인
df_all[df_all.duplicated(['지점명'], keep=False)]

,브랜드명,지점명,시도명,주소
7,파스쿠찌,구미인동점,경상북도,경북 구미시 인동남길 130
447,파스쿠찌,구미인동점,경상북도,경상북도 구미시 인동남길 130


In [25]:
# 중복 행 제거
df_all = df_all.drop_duplicates(['지점명'])
df_all

,브랜드명,지점명,시도명,주소
0,파스쿠찌,고양향동스타허브점,경기도,"경기도 고양시 덕양구 꽃마을로 64, DMC스타허브플러스 1층"
1,파스쿠찌,대전중앙로점,대전,"대전광역시 중구 중앙로 126, 1층 (대흥동)"
2,파스쿠찌,세종다정중앙점,세종시특별자치시,세종특별시 다정중앙로 46
3,파스쿠찌,군산디오션시티점,전라북도,전라북도 군산시 백릉로 145
4,파스쿠찌,홈플러스아시아드점,부산,부산 연제구 거제동 1208 홈플러스
...,...,...,...,...
517,파스쿠찌,수원AK점,경기도,경기도수원시 팔달구 매산로1가 수원역사 AK PLAZA 3층
518,파스쿠찌,롯데마산점,경상남도,경상남도 창원시 마산합포구 신포동2가 28
519,파스쿠찌,광안리점,부산,부산시 수영구 민락동 179-2 파스쿠찌 광안리점
520,파스쿠찌,테크노마트점,서울,서울시 광진구 구의동 546-4 테크노마트 지하 D-10호 파스쿠찌 테크노마트점


In [26]:
df_all.nunique()

브랜드명      1
지점명     521
시도명      17
주소      521
dtype: int64

In [27]:
# 521개 매장 정보 excel 파일로 저장
df_all.to_excel('pascucci.xlsx', index=False)